In [1]:
import numpy as np
import pandas as pd
import psycopg2

In [14]:
%store -r all_formatted_data
%store -r names

In [15]:
char_to_remove = [str(i) for i in range(10)]
specials_to_remove = ['*', '/', '**', '***', '(', '[', ')', ']', '&']
yelenas_to_remove = ['did', 'doing', 'didnt', 'need', 'could', 'couldnt', '?']
emphasis_to_remove = ['emphasis', 'Emphasis', 'Focus', 'Condition', 'recon', 'recondition', 'Recondition', 'wedding']

def passes_filters(exercise):
    if any(char in exercise for char in char_to_remove):
        return False
    if any(char in exercise for char in specials_to_remove):
        return False
    if any(char in exercise for char in yelenas_to_remove):
        return False
    if any(char in exercise for char in emphasis_to_remove):
        return False
    return True

In [16]:
def convert_to_int(data):
    try:
        for item in data:
            item['weight'] = int(float(item['weight']))
            item['sets'] = int(float(item['sets']))
            item['reps'] = int(float(item['reps']))
    except ValueError:
        return None

    return data

In [17]:
def unique_pairs(data):
    unique_pairs = {}

    for item in data:
        key = (item['exercise'], tuple([item['weight'], item['sets'], item['reps']]))

        if key not in unique_pairs:
            unique_pairs[key] = item

    return list(unique_pairs.values())

In [18]:
cleaned_dict={}
for client in all_formatted_data:
    client_data=[]
    for workout in all_formatted_data[client]:
        for rows in all_formatted_data[client][workout]:
            client_data.append(rows)
    cleaned_dict[client]=client_data
            

In [19]:
for i in cleaned_dict:
    print(i)

Dimitri
Masha
Misha
Anya
Arne
Max
Dino
Test
Ilya
Igor
Nick_Mary
Daniel Murphy
Yelena
Shawn


In [20]:
filtered_dict={}
for client in cleaned_dict:
    flitered_client=[]
    try:
        for item in cleaned_dict[client]:
            if passes_filters(item['exercise']):
                flitered_client.append(item)
    except:
        pass
    filtered_dict[client]=flitered_client

In [21]:
for i in filtered_dict:
    print(i)

Dimitri
Masha
Misha
Anya
Arne
Max
Dino
Test
Ilya
Igor
Nick_Mary
Daniel Murphy
Yelena
Shawn


In [22]:
for client in filtered_dict:
    converted_data = convert_to_int(filtered_dict[client])
    filtered_dict[client] = converted_data


In [23]:
for i in filtered_dict:
    print(i)

Dimitri
Masha
Misha
Anya
Arne
Max
Dino
Test
Ilya
Igor
Nick_Mary
Daniel Murphy
Yelena
Shawn


In [24]:
for client in filtered_dict:
    unique_data = unique_pairs(converted_data)
    filtered_dict[client]=unique_data

## Some houskeeping on the keys before entering it into database

In [25]:
filtered_dict.pop("Test", None)

[{'exercise': 'J-Jacks', 'weight': 1, 'sets': 2, 'reps': 1},
 {'exercise': 'LB Warm Up', 'weight': 1, 'sets': 3, 'reps': 1},
 {'exercise': 'Pause Squats', 'weight': 5, 'sets': 3, 'reps': 10},
 {'exercise': 'Static Lunge', 'weight': 7, 'sets': 3, 'reps': 10},
 {'exercise': 'SL RDL', 'weight': 10, 'sets': 3, 'reps': 10},
 {'exercise': 'SL Glute Bridges', 'weight': 1, 'sets': 3, 'reps': 10},
 {'exercise': 'Hamstring Planks', 'weight': 1, 'sets': 3, 'reps': 30},
 {'exercise': 'Seated Piriformis and Butterfly',
  'weight': 1,
  'sets': 3,
  'reps': 30},
 {'exercise': 'Hamstring and Calf', 'weight': 1, 'sets': 1, 'reps': 1},
 {'exercise': 'Ankle Walks', 'weight': 1, 'sets': 3, 'reps': 1},
 {'exercise': 'SL RDL', 'weight': 8, 'sets': 3, 'reps': 10},
 {'exercise': 'SL Glute Bridges', 'weight': 7, 'sets': 3, 'reps': 10},
 {'exercise': 'SL Calf Raises', 'weight': 1, 'sets': 3, 'reps': 10},
 {'exercise': 'SL RDL', 'weight': 1, 'sets': 3, 'reps': 10},
 {'exercise': 'SL Sit to Stands', 'weight': 1,

In [26]:
filtered_dict.pop("Nick_Mary", None)

[{'exercise': 'J-Jacks', 'weight': 1, 'sets': 2, 'reps': 1},
 {'exercise': 'LB Warm Up', 'weight': 1, 'sets': 3, 'reps': 1},
 {'exercise': 'Pause Squats', 'weight': 5, 'sets': 3, 'reps': 10},
 {'exercise': 'Static Lunge', 'weight': 7, 'sets': 3, 'reps': 10},
 {'exercise': 'SL RDL', 'weight': 10, 'sets': 3, 'reps': 10},
 {'exercise': 'SL Glute Bridges', 'weight': 1, 'sets': 3, 'reps': 10},
 {'exercise': 'Hamstring Planks', 'weight': 1, 'sets': 3, 'reps': 30},
 {'exercise': 'Seated Piriformis and Butterfly',
  'weight': 1,
  'sets': 3,
  'reps': 30},
 {'exercise': 'Hamstring and Calf', 'weight': 1, 'sets': 1, 'reps': 1},
 {'exercise': 'Ankle Walks', 'weight': 1, 'sets': 3, 'reps': 1},
 {'exercise': 'SL RDL', 'weight': 8, 'sets': 3, 'reps': 10},
 {'exercise': 'SL Glute Bridges', 'weight': 7, 'sets': 3, 'reps': 10},
 {'exercise': 'SL Calf Raises', 'weight': 1, 'sets': 3, 'reps': 10},
 {'exercise': 'SL RDL', 'weight': 1, 'sets': 3, 'reps': 10},
 {'exercise': 'SL Sit to Stands', 'weight': 1,

In [28]:
old_key = 'Daniel Murphy'
new_key = 'Daniel'

if old_key in filtered_dict:
    filtered_dict[new_key] = filtered_dict[old_key]
    del filtered_dict[old_key]

In [29]:
old_key = 'Max'
new_key = 'Abelev'

if old_key in filtered_dict:
    filtered_dict[new_key] = filtered_dict[old_key]
    del filtered_dict[old_key]

In [30]:
for i in filtered_dict:
    print (i)

Dimitri
Masha
Misha
Anya
Arne
Dino
Ilya
Igor
Yelena
Shawn
Daniel
Abelev


In [26]:
for client_id, client_data in filtered_dict.items():
    for data in client_data:
        print(data)

{'exercise': 'J-Jacks', 'weight': 1, 'sets': 2, 'reps': 1}
{'exercise': 'LB Warm Up', 'weight': 1, 'sets': 3, 'reps': 1}
{'exercise': 'Pause Squats', 'weight': 5, 'sets': 3, 'reps': 10}
{'exercise': 'Static Lunge', 'weight': 7, 'sets': 3, 'reps': 10}
{'exercise': 'SL RDL', 'weight': 10, 'sets': 3, 'reps': 10}
{'exercise': 'SL Glute Bridges', 'weight': 1, 'sets': 3, 'reps': 10}
{'exercise': 'Hamstring Planks', 'weight': 1, 'sets': 3, 'reps': 30}
{'exercise': 'Seated Piriformis and Butterfly', 'weight': 1, 'sets': 3, 'reps': 30}
{'exercise': 'Hamstring and Calf', 'weight': 1, 'sets': 1, 'reps': 1}
{'exercise': 'Ankle Walks', 'weight': 1, 'sets': 3, 'reps': 1}
{'exercise': 'SL RDL', 'weight': 8, 'sets': 3, 'reps': 10}
{'exercise': 'SL Glute Bridges', 'weight': 7, 'sets': 3, 'reps': 10}
{'exercise': 'SL Calf Raises', 'weight': 1, 'sets': 3, 'reps': 10}
{'exercise': 'SL RDL', 'weight': 1, 'sets': 3, 'reps': 10}
{'exercise': 'SL Sit to Stands', 'weight': 1, 'sets': 3, 'reps': 10}
{'exercise'

In [33]:
single_client=filtered_dict['Ilya']

In [70]:
filtered_dict.keys()

dict_keys(['Dimitri', 'Masha', 'Misha', 'Anya', 'Arne', 'Dino', 'Ilya', 'Igor', 'Yelena', 'Shawn', 'Daniel', 'Abelev'])

In [71]:
conn=psycopg2.connect()

cursor=conn.cursor()



# Iterate through the clients and their data in the filtered_dict
for client, client_data in filtered_dict.items():
    for data in client_data:
        exercise = data['exercise']
        sets = data['sets']
        reps = data['reps']
        weight = data['weight']
        
        
        # Check if the exercise exists in the exercise table
        cursor.execute("""
        SELECT id FROM exercises WHERE exercise = %s
        """,(exercise,))
        exercise_id = cursor.fetchone()
        
        # Insert the exercise into the exercise table if it doesn't exist
        if exercise_id is None:
            cursor.execute("""
            INSERT INTO exercises (exercise)
            VALUES (%s)
            RETURNING id
            """,(exercise,))
            exercise_id = cursor.fetchone()

        cursor.execute("""
        INSERT INTO training_data (client_id, exercise_id, sets, reps, weight)
        VALUES (
            (SELECT id FROM client WHERE name = %s),%s,%s, %s, %s)
        """,(client, exercise_id, sets, reps, weight))


# Commit the changes and close the connection
conn.commit()
conn.close()

In [35]:
conn.close()

In [38]:
# Connect to the PostgreSQL database
conn = psycopg2.connect()

cursor = conn.cursor()

In [72]:
conn=psycopg2.connect()

cursor=conn.cursor()

cursor.execute("""
SELECT client_id, exercise_id, weight, sets, reps
FROM workout_exercises;
""")
workout_data = cursor.fetchall()
conn.close()

In [73]:
workout_data

[(4, 2, 6, 3, 30),
 (4, 48, 6, 3, 8),
 (4, 91, 65, 3, 8),
 (4, 8, 11, 3, 10),
 (4, 20, 11, 3, 10),
 (4, 37, 11, 3, 8),
 (15, 2, 1, 4, 30),
 (15, 60, 10, 3, 10),
 (15, 5, 1, 3, 8),
 (15, 6, 1, 3, 30),
 (15, 61, 1, 3, 30),
 (15, 8, 1, 3, 10),
 (4, 2, 11, 3, 30),
 (4, 48, 11, 3, 8),
 (4, 91, 70, 3, 8),
 (19, 149, 20, 3, 15),
 (19, 99, 1, 3, 15),
 (19, 41, 20, 3, 15),
 (19, 52, 20, 3, 15),
 (19, 31, 1, 3, 30),
 (19, 32, 1, 3, 30),
 (19, 7, 1, 3, 30),
 (4, 2, 1, 3, 30),
 (4, 60, 110, 3, 10),
 (4, 50, 30, 1, 20),
 (4, 94, 210, 3, 8),
 (4, 6, 1, 3, 30),
 (19, 149, 10, 3, 15),
 (19, 118, 1, 3, 15),
 (19, 53, 20, 3, 10),
 (19, 93, 1, 3, 8),
 (19, 21, 20, 3, 8),
 (19, 61, 1, 3, 30),
 (23, 119, 1, 1, 9),
 (23, 120, 1, 1, 9),
 (23, 35, 1, 3, 60),
 (23, 7, 1, 3, 30),
 (23, 250, 15, 3, 10),
 (23, 251, 10, 3, 10),
 (23, 252, 1, 3, 10),
 (23, 253, 1, 3, 10),
 (23, 2, 1, 3, 30),
 (8, 2, 1, 3, 30),
 (8, 147, 20, 3, 10),
 (8, 267, 20, 3, 8),
 (8, 53, 20, 3, 8),
 (8, 180, 1, 3, 30),
 (8, 33, 1, 3, 30),
 (

In [74]:
unique_data = set()
sanitized_data = []

for row in workout_data:
    key = (row[0], row[1], row[2], row[3], row[4])
    if key not in unique_data:
        unique_data.add(key)
        sanitized_data.append(row)


In [75]:
sanitized_data

[(4, 2, 6, 3, 30),
 (4, 48, 6, 3, 8),
 (4, 91, 65, 3, 8),
 (4, 8, 11, 3, 10),
 (4, 20, 11, 3, 10),
 (4, 37, 11, 3, 8),
 (15, 2, 1, 4, 30),
 (15, 60, 10, 3, 10),
 (15, 5, 1, 3, 8),
 (15, 6, 1, 3, 30),
 (15, 61, 1, 3, 30),
 (15, 8, 1, 3, 10),
 (4, 2, 11, 3, 30),
 (4, 48, 11, 3, 8),
 (4, 91, 70, 3, 8),
 (19, 149, 20, 3, 15),
 (19, 99, 1, 3, 15),
 (19, 41, 20, 3, 15),
 (19, 52, 20, 3, 15),
 (19, 31, 1, 3, 30),
 (19, 32, 1, 3, 30),
 (19, 7, 1, 3, 30),
 (4, 2, 1, 3, 30),
 (4, 60, 110, 3, 10),
 (4, 50, 30, 1, 20),
 (4, 94, 210, 3, 8),
 (4, 6, 1, 3, 30),
 (19, 149, 10, 3, 15),
 (19, 118, 1, 3, 15),
 (19, 53, 20, 3, 10),
 (19, 93, 1, 3, 8),
 (19, 21, 20, 3, 8),
 (19, 61, 1, 3, 30),
 (23, 119, 1, 1, 9),
 (23, 120, 1, 1, 9),
 (23, 35, 1, 3, 60),
 (23, 7, 1, 3, 30),
 (23, 250, 15, 3, 10),
 (23, 251, 10, 3, 10),
 (23, 252, 1, 3, 10),
 (23, 253, 1, 3, 10),
 (23, 2, 1, 3, 30),
 (8, 2, 1, 3, 30),
 (8, 147, 20, 3, 10),
 (8, 267, 20, 3, 8),
 (8, 53, 20, 3, 8),
 (8, 180, 1, 3, 30),
 (8, 33, 1, 3, 30),
 (

In [76]:
conn=psycopg2.connect()

cursor=conn.cursor()

for row in sanitized_data:
    cursor.execute("""
    INSERT INTO training_data (client_id, exercise_id, weight, sets, reps)
            VALUES (%s, %s, %s, %s, %s);
    """, (row[0], row[1], row[2], row[3], row[4]))


conn.commit()
conn.close()

### Getting the unique_inputs and unique_outputs for my tokenize and pad function from new query now that data is added to databse

In [84]:
conn=psycopg2.connect()
cursor=conn.cursor()
cursor.execute("""
        SELECT 
            c.name AS client_name, 
            e.exercise AS exercise_name, 
            td.weight, 
            td.sets, 
            td.reps
        FROM training_data td
        LEFT JOIN client c ON td.client_id = c.id
        LEFT JOIN exercises e ON td.exercise_id = e.id;
        """)
all_workout_data_json = cursor.fetchall()

conn.close()

In [93]:
DTR_training_data=np.asarray(all_workout_data_json)
%store DTR_training_data

Stored 'DTR_training_data' (ndarray)


In [88]:
test=pd.DataFrame(workout_data)

In [89]:
test

,0,1,2,3,4
0,Masha,Banded walks,3,3,1
1,Masha,Clam Shells,3,3,30
2,Masha,Abductions,3,3,30
3,Masha,Squat Thrusters,10,3,15
4,Masha,SL Bridges,1,3,30
...,...,...,...,...,...
2432,Masha,Abductions,1,3,30
2433,Masha,Pause Squats,1,3,15
2434,Masha,Hammy Planks,1,1,1
2435,Masha,Piegon Pose,1,1,1


In [44]:
import tensorflow as tf

In [52]:
def filter_data(inputs, outputs):
    #Filtering out exercises that don't pass the string filters
    new_inputs = []
    new_outputs = []
    for i, exercise in enumerate(inputs):
        if passes_filters(exercise):
            new_inputs.append(exercise)
            new_outputs.append(outputs[i])

    inputs = new_inputs
    outputs = new_outputs

    # Making sure all values are integers
    converted_outputs = []
    new_inputs = []

    for i, arr in enumerate(outputs):
        converted_arr = convert_to_int(arr)
        if converted_arr is not None:
            converted_outputs.append(converted_arr)
            new_inputs.append(inputs[i])

    inputs = new_inputs
    outputs = converted_outputs

    return inputs, outputs

In [54]:
def convert_to_int(arr):
    try:
        return np.array([int(float(elem)) for elem in arr], dtype=int)
    except ValueError:
        return None

In [56]:
inputs, outputs=filter_data(workout_data[:,1], workout_data[:,2:])

In [50]:
def tokenize_and_pad(unique_inputs, unique_outputs):
    # Tokenize inputs
    tokenizer=tf.keras.preprocessing.text.Tokenizer
    pad_sequences=tf.keras.preprocessing.sequence.pad_sequences

    input_tokenizer = tokenizer(char_level=False, filters='', lower=False)
    input_tokenizer.fit_on_texts(unique_inputs)
    input_sequences = input_tokenizer.texts_to_sequences(unique_inputs)

    # Pad sequences
    input_seq_padded = pad_sequences(input_sequences, padding='pre')

    #Changing outputs to 2D Array
    outputs=np.asarray(unique_outputs)
    outputs

    return input_tokenizer, pad_sequences, input_seq_padded, outputs

In [57]:
tokenize_and_pad(inputs, outputs)

(<keras.preprocessing.text.Tokenizer at 0x28814c130>,
 <function keras.utils.data_utils.pad_sequences(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)>,
 array([[ 0,  0,  0,  0,  0, 82],
        [ 0,  0,  0, 83, 84, 74],
        [ 0,  0,  0,  0, 34, 11],
        ...,
        [ 0,  0,  0,  0,  0, 25],
        [ 0,  3, 36,  9,  6,  8],
        [ 0,  0,  1,  2,  4,  5]], dtype=int32),
 array([[ 1,  2,  1],
        [ 1,  3,  1],
        [ 5,  3, 10],
        ...,
        [ 1,  3, 15],
        [20,  3, 10],
        [20,  3, 10]]))

In [91]:
test[test[1]=='Back Squats']

,0,1,2,3,4
1391,Masha,Back Squats,90,3,5
1458,Masha,Back Squats,95,3,5
1764,Jared,Back Squats,185,3,10
1815,Yelena,Back Squats,75,4,5
1913,Yelena,Back Squats,65,4,5
1980,Nat,Back Squats,90,4,5
2020,Yelena,Back Squats,55,4,5
2055,Masha,Back Squats,100,3,5
2145,Yelena,Back Squats,80,4,5
2183,Masha,Back Squats,85,3,5


In [63]:
l=[]
for name in filtered_dict:
    for row in filtered_dict[name]:
        l.append(row)

In [64]:
len(l)

1488